In [1]:
import pandas as pd
from Project import collector
import sys

In [2]:
sys.stdout = open("ComDump", "w", encoding="utf-8")
data = collector.single_run_collector(with_communication=True, seed=42)
sys.stdout.flush()

ValueError: Knowledge base is trying to remove a breeze.

In [ ]:
import re
with open("ComDump", "r", encoding="utf-8") as file:
    lines = file.readlines()
    for i in range(len(lines)):
        if re.search("(The request is completed)|(The negotiation has reached an agreement)", lines[i]) is not None:
            print(i)